### Convolution, filter, and pooling

In [1]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as dsets
from torch.autograd import Variable

In [2]:
train_dataset = dsets.MNIST(root='./data',
                           train=True,
                           transform=transforms.ToTensor(),
                           download=True)

test_dataset = dsets.MNIST(root='./data',
                          train=False,
                          transform=transforms.ToTensor())

In [3]:
batch_size = 100
n_iters = 3000
num_epochs = n_iters / (len(train_dataset)/batch_size)
num_epochs = int(num_epochs)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                          batch_size=batch_size,
                                          shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                         batch_size=batch_size,
                                         shuffle=False)

In [4]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        
        # input is (28,28) grayscale image (single channel)
        self.cnn1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=5, stride=1, padding=2)
        self.relu1 = nn.ReLU()
        
        # num of output can be computed by (Width/KernelSize, Height/KernelSize) = (14,14)
        self.maxpool1 = nn.MaxPool2d(kernel_size=2)
        #self.maxpool1 = nn.AvgPool2d(kernel_size=2)
        
        # input is (14,14) images x16 = (16,14,14)
        self.cnn2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=5, stride=1, padding=2)
        self.relu2 = nn.ReLU()
        
        # num of output can be computed by (Width/KernelSize, Height/KernelSize) = (7,7)
        self.maxpool2 = nn.MaxPool2d(kernel_size=2)
        #self.maxpool2 = nn.AvgPool2d(kernel_size=2)
        
        self.fc1 = nn.Linear(32 * 7 * 7, 10)
        
    def forward(self, x):
        out = self.cnn1(x)
        out = self.relu1(out)
        
        out = self.maxpool1(out)
        
        out = self.cnn2(out)
        out = self.relu2(out)
        
        out = self.maxpool2(out)
        
        out = out.view(out.size(0), -1)
        
        out = self.fc1(out)
        
        return out

In [5]:
model = CNN()

# GPU acceleration
if torch.cuda.is_available():
    model.cuda()

In [6]:
criterion = nn.CrossEntropyLoss()

In [7]:
lr = 0.01
optimizer = torch.optim.SGD(model.parameters(), lr=lr)

In [8]:
iter = 0
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        
        # GPU acceleration
        if torch.cuda.is_available():
            images = Variable(images.cuda())
            labels = Variable(labels.cuda())
        else:
            images = Variable(images)
            labels = Variable(labels)
            
        optimizer.zero_grad()
        
        outputs = model(images)
        
        loss = criterion(outputs, labels)
        
        loss.backward()
        
        optimizer.step()
        
        iter += 1
        
        if iter % 500 == 0:
            correct = 0
            total = 0
            
            for images, labels in test_loader:
                
                # GPU acceleration
                if torch.cuda.is_available():
                    images = Variable(images.cuda())
                else:
                    images = Variable(images)
                
                outputs = model(images)
                
                _, predicted = torch.max(outputs.data, 1)
                
                total += labels.size(0)
                
                if torch.cuda.is_available():
                    correct += (predicted.cpu() == labels.cpu()).sum()
                else:
                    correct += (predicted == labels).sum()
                
            accuracy = 100 * correct/total
            print('Iteration: {}. Loss: {}. Accuracy: {}'.format(iter, loss.data[0], accuracy))

/home/takashi/.virtualenvs/rl/lib/python3.5/site-packages/ipykernel_launcher.py:49: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


Iteration: 500. Loss: 0.3746863901615143. Accuracy: 89
Iteration: 1000. Loss: 0.28346461057662964. Accuracy: 92
Iteration: 1500. Loss: 0.1071692407131195. Accuracy: 94
Iteration: 2000. Loss: 0.23203854262828827. Accuracy: 95
Iteration: 2500. Loss: 0.09685149043798447. Accuracy: 96
Iteration: 3000. Loss: 0.13424892723560333. Accuracy: 96


In [9]:
import os
res = os.popen('python --version')
print(res.readline())

Python 3.5.2

